In [2]:
using Crystalline
using MPBUtils
using Crystalline: TEST_αβγs
using Crystalline: TEST_αβγ
using Brillouin 
using Crystalline: dot
using Crystalline: norm

In [4]:
?normscale!

search: normscale! normscale



```
normscale!(flat::ModulatedFourierLattice, expon::Real, Gs::Union{ReciprocalBasis, Nothing} = nothing) --> ModulatedFourierLattice
```

In-place equivalent of `normscale`: changes `flat`.


In [1]:
using Pkg; 
Pkg.activate("/home/gridsan/aligho/Github_Projects/Crystalline.jl")

In [3]:
write_dir = (@__DIR__)
if !isdefined(Main, :INVERSION_PLANEGROUPS)
    const INVERSION_PLANEGROUPS = Bool[0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1]
end
has_inversion(sgnum) = INVERSION_PLANEGROUPS[sgnum]


has_inversion (generic function with 1 method)

In [4]:
# Code from before normscale was introduced into Crystalline
function my_normscale!(flat::ModulatedFourierLattice, expon::Real, Gs::ReciprocalBasis{D}) where D
    if !iszero(expon)
        orbits = getorbits(flat)
        @inbounds for i in eachindex(orbits)
            rescale_factor = norm(sum(Gs.*first(orbits[i])))^expon
            rescale_factor == zero(rescale_factor) && continue # for G = [0,0,0] case
            flat.orbitcoefs[i] ./= rescale_factor
        end
    end
    return flat
end

my_normscale! (generic function with 1 method)

In [5]:
Gs = reciprocalbasis(directbasis(17, 2))

2-element ReciprocalBasis{2} with indices SOneTo(2):
 [6.283185307179586, 3.6275987284684335]
 [-0.0, 7.2551974569368705]

In [6]:
function make_planegroup_lattice(sgnum, ::Val{D}, cntr::Char, Gs::ReciprocalBasis{D};
                maxGs=ntuple(_->6, Val(D)), expon::Real=1.25) where D
    flat = my_normscale!(modulate(levelsetlattice(sgnum, D, maxGs)), expon, Gs)
    deleteat!(flat.orbits, 1), deleteat!(flat.orbitcoefs, 1) # trivial constant G=0 term
    map!(flat.orbitcoefs, flat.orbitcoefs) do coefs
        if has_inversion(sgnum)
            map!(x -> round(real(x), digits=4), coefs, coefs)
        else
            map!(x -> round(x, digits=4), coefs, coefs) 
        end
    end
    pflat = primitivize(flat, cntr)
    return pflat
end 

make_planegroup_lattice (generic function with 1 method)

In [ ]:
D=2
epsout = 1.0
has_tr = true; res = 64; nbands = 40; interpolatekvs = 10
write_to_file = true 

sgnums = [2, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17]  # Only look at three spacegroups- ones with nontrivial stable topology
for sgnum in sgnums
    println(sgnum)
    cntr = centering(sgnum, D)
    brs  = bandreps(sgnum, D, timereversal=has_tr)
    lgs  = Crystalline.matching_littlegroups(brs, Val(D))
    plgs = primitivize.(lgs, #=modw=# false)
    kvs = brs.kvs
    converted_kvs = [convert(Vector{Real}, kv(TEST_αβγs[2])) for kv in kvs]
    ids = 1:1000
    for id in ids
        (iszero(id%10) &&  println(id))
        Rs = directbasis(sgnum, Val(D))
        Gs = reciprocalbasis(Rs)
        #println(Gs)
        pRs = primitivize(Rs, cntr)
        kpaths = interpolate(irrfbz_path(sgnum, pRs, 2), 30).kpaths
        allkpaths = Vector{Vector{Float64}}()
        for kpath in kpaths
            for k in kpath
                push!(allkpaths, k)
            end
        end
        pflat = make_planegroup_lattice(sgnum, Val(D), cntr, Gs)
        
        filling = rand(0.25:0.05:0.75)
        
        for (dielectric_enum, epsin) in enumerate((16))#enumerate((8, 12, 16))
            for runtype in ["te", "tm"]
                filename = mpb_calcname(D, sgnum, id, res, runtype)
                filepath_dispersion = joinpath(write_dir, "dispersions/input/"*filename*".sh")
                filepath_symeig = joinpath(write_dir, "symeigs/input/"*filename*".sh")
                write_to_file || continue
                open(filepath_dispersion, "w") do io
                    prepare_mpbcalc!(io, sgnum, pflat, pRs, filling, epsin, epsout, runtype; #Test that what we're giving to prepare_mpbcalc! is correct
                                        res=res, kvecs=allkpaths, id = id, nbands=nbands)
                    αβγ = length(TEST_αβγ) == D ? TEST_αβγ : TEST_αβγ[1:D]	
                    test_kvs = (map(lg->kvec(lg)(αβγ), lgs))
                end
                open(filepath_symeig, "w") do io
                    prepare_mpbcalc!(io, sgnum, pflat, pRs, filling, epsin, epsout, runtype;
                                        res=res, lgs=plgs, id = id, nbands=nbands)
                    αβγ = length(TEST_αβγ) == D ? TEST_αβγ : TEST_αβγ[1:D]	
                    test_kvs = (map(lg->kvec(lg)(αβγ), lgs))
                end
            end
        end
    end
end

2
10
20
30
40
50
60
70
80


In [11]:
D=2
epsout = 1.0
has_tr = true; res = 64; nbands = 40; interpolatekvs = 10
write_to_file = true 

cntr = centering(sgnum, D)
brs  = bandreps(sgnum, D, timereversal=has_tr)
lgs  = Crystalline.matching_littlegroups(brs, Val(D))
plgs = primitivize.(lgs, #=modw=# false)
kvs = brs.kvs
converted_kvs = [convert(Vector{Real}, kv(TEST_αβγs[2])) for kv in kvs]
ids = 1:1000
for id in ids
    (iszero(id%10) &&  println(id))
    Rs = directbasis(sgnum, Val(D))
    Gs = reciprocalbasis(Rs)
    #println(Gs)
    pRs = primitivize(Rs, cntr)
    kpaths = interpolate(irrfbz_path(sgnum, pRs, 2), 30).kpaths
    allkpaths = Vector{Vector{Float64}}()
    for kpath in kpaths
        for k in kpath
            push!(allkpaths, k)
        end
    end
    pflat = make_planegroup_lattice(sgnum, Val(D), cntr, Gs)

    filling = rand(0.25:0.05:0.75)

    for (dielectric_enum, epsin) in enumerate((8, 12, 16))
        for runtype in ["te", "tm"]
            filename = mpb_calcname(D, sgnum, id, res, runtype)
            filepath_dispersion = joinpath(write_dir, "dispersions/input/"*filename*".sh")
            filepath_symeig = joinpath(write_dir, "symeigs/input/"*filename*".sh")
            write_to_file || continue
            open(filepath_dispersion, "w") do io
                prepare_mpbcalc!(io, sgnum, pflat, pRs, filling, epsin, epsout, runtype; #Test that what we're giving to prepare_mpbcalc! is correct
                                    res=res, kvecs=allkpaths, id = id + (dielectric_enum-1)*1000, nbands=nbands)
                αβγ = length(TEST_αβγ) == D ? TEST_αβγ : TEST_αβγ[1:D]	
                test_kvs = (map(lg->kvec(lg)(αβγ), lgs))
            end
            open(filepath_symeig, "w") do io
                prepare_mpbcalc!(io, sgnum, pflat, pRs, filling, epsin, epsout, runtype;
                                    res=res, lgs=plgs, id = id + (dielectric_enum-1)*1000, nbands=nbands)
                αβγ = length(TEST_αβγ) == D ? TEST_αβγ : TEST_αβγ[1:D]	
                test_kvs = (map(lg->kvec(lg)(αβγ), lgs))
            end
        end
    end
end


LoadError: UndefVarError: sgnum not defined

In [5]:
?prepare_mpbcalc!

search: prepare_mpbcalc! prepare_mpbcalc



```
prepare_mpbcalc!(...)
```

Formats a set of parameters that uniquely specify an MPB calculation, given a  space group number `sgnum`, a Fourier lattice `flat`, a DirectBasis `Rs`, a filling fraction `filling` for `flat`, interior and exterior (above, below the contour) permittivities `εin` and `εout`, as well as a list of k-vectors `kvecs`, an  identifying tag `id` (to label the calculation for book-keeping purposes), a  resolution for the MPB calculation `res`, and a selection of calculation type `runtype` ("all", "te", or "tm"). The results are written to requested IO `io`.

Our preferred choice is to write these parameters to a bash file, with a name generated by the `mpb_calcname(...)` method.

The options are expected to be fed to the `fourier-lattice.ctl` file, e.g. through a bash script of the following kind:

```sh
    IFS=$'\n'; # stop command-substitutions from word-splitting at space

    PATH_TO_MPB_EXECUTABLE \
        (cat ${calcname}.sh)
        ctl/fourier-lattice.ctl 2>&1 | tee logs/${calcname}.log
        
    unset IFS; # restore usual command-substitution word-splitting practice
```

where `PATH_TO_MPB_EXECUTABLE` is the path to the MPB executable. Locally, in `mpb-ctl` we have a file `run-fourier-lattice.sh` which performs the  above, with `calcname` specified as an input parameter (assumed to be a subfolder `/input/`).


In [10]:
methods(prepare_mpbcalc!)

# 5 methods for generic function "prepare_mpbcalc!":
[1] prepare_mpbcalc!(io::IO, sgnum::Integer, flat::Crystalline.AbstractFourierLattice{D}, Rs::AbstractVector{var"#s13"} where var"#s13"<:(AbstractVector{T} where T)) where D in MPBUtils at /home/gridsan/aligho/.julia/packages/MPBUtils/64Wit/src/export2mpb.jl:107
[2] prepare_mpbcalc!(io::IO, sgnum::Integer, flat::Crystalline.AbstractFourierLattice{D}, Rs::AbstractVector{var"#s14"} where var"#s14"<:(AbstractVector{T} where T), filling::Union{Nothing, Real}) where D in MPBUtils at /home/gridsan/aligho/.julia/packages/MPBUtils/64Wit/src/export2mpb.jl:107
[3] prepare_mpbcalc!(io::IO, sgnum::Integer, flat::Crystalline.AbstractFourierLattice{D}, Rs::AbstractVector{var"#s15"} where var"#s15"<:(AbstractVector{T} where T), filling::Union{Nothing, Real}, εin::Real) where D in MPBUtils at /home/gridsan/aligho/.julia/packages/MPBUtils/64Wit/src/export2mpb.jl:107
[4] prepare_mpbcalc!(io::IO, sgnum::Integer, flat::Crystalline.AbstractFourierLattice{D}, Rs::AbstractVector{var"#s16"} where var"#s16"<:(AbstractVector{T} where T), filling::Union{Nothing, Real}, εin::Real, εout::Real) where D in MPBUtils at /home/gridsan/aligho/.julia/packages/MPBUtils/64Wit/src/export2mpb.jl:107
[5] prepare_mpbcalc!(io::IO, sgnum::Integer, flat::Crystalline.AbstractFourierLattice{D}, Rs::AbstractVector{var"#s17"} where var"#s17"<:(AbstractVector{T} where T), filling::Union{Nothing, Real}, εin::Real, εout::Real, runtype::String; id, res, kvecs, lgs, nbands, isoval) where D in MPBUtils at /home/gridsan/aligho/.julia/packages/MPBUtils/64Wit/src/export2mpb.jl:107

In [1]:
transpose([0, 1])

1×2 transpose(::Vector{Int64}) with eltype Int64:
 0  1

In [2]:
norm(first(orbits[i])'*Gs)

1×2 Matrix{Int64}:
 0  1